In [266]:
import requests
from bs4 import BeautifulSoup
import requests
from paperview.retrieval.biorxiv_api import (
    Article
)
from paperview.retrieval import pdf_extraction
import tempfile
from io import StringIO
from PIL import Image
import seaborn as sns
import pandas as pd
from io import StringIO

import pdfplumber

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [388]:
url = 'https://www.biorxiv.org/content/10.1101/2022.07.21.500746v1'
# url = 'https://www.biorxiv.org/content/10.1101/456574v1'
# url = 'https://www.biorxiv.org/content/10.1101/2022.12.21.521407v1?rss=1'
# url = 'https://www.biorxiv.org/content/10.1101/2022.09.12.507457v1.full'
# url = 'https://www.biorxiv.org/content/10.1101/386375v1.full'
# url = 'https://www.biorxiv.org/content/10.1101/2022.07.20.500837v1?rss=1'
# url = 'https://www.biorxiv.org/content/10.1101/180125v2.full'
# url = 'https://www.biorxiv.org/content/10.1101/2022.12.23.521820v1?rss=1'
article = Article.from_content_page_url(url, prog_bar=True)

100%|██████████| 36/36 [00:20<00:00,  1.79it/s]


In [371]:
article.article_detail


ArticleDetail(
    title='Determinants of Motor Neuron Functional Subtypes Important for Locomotor Speed',
    authors=["D'Elia, K. P.", 'Hameedy, H.', 'Goldblatt, D.', 'Frazel, P.', 'Kriese, M.', 'Zhu, Y.', 'Hamling, K. R.', 'Kawakami, K.', 'Liddelow, S. A.', 'Schoppik, D.', 'Dasen, J. S.'],
    date='2022-12-26',
    category='neuroscience',
    doi='10.1101/2022.12.23.521820',
    author_corresponding='Jeremy S Dasen',
    author_corresponding_institution='NYU Grossman School of Medicine',
    version='1',
    type='new results',
    license='cc_by',
    abstract='Locomotion requires precise control of the strength and speed of muscle contraction and is achieved by recruiting functionally-distinct subtypes of motor neurons (MNs). MNs are essential to movement and differentially susceptible in disease, but little is known about how MNs acquire functional subtype-specific molecular features during development. Using single cell RNA profiling in larval zebrafish, we identify novel and

In [369]:
article.display_overview()

In [268]:
lines = article.data['lines']

In [269]:
def add_paragraph_number(lines: pd.DataFrame) -> pd.DataFrame:
    """Add a paragraph number to each line of text.
    Paragraphs are defined as groups of lines that are separated by at least 1.5 times the average line height.
    """
    lines = lines.copy()
    
    # filter out biorxiv watermark lines
    lines = lines.query('is_biorxiv_watermark_line == False')
    
    # filter out single word lines
    lines = lines.query('num_words > 1')
    
    lines['paragraph_number'] = 0
    lines['line_height'] = lines['bottom_abs'] - lines['top_abs']
    avg_line_height = lines['line_height'].mean()
    lines['paragraph_number'] = (lines['dist_to_next_line'] > 1.5 * avg_line_height).cumsum()
    return lines

def lines_to_paragraphs(lines: pd.DataFrame) -> pd.DataFrame:
    """Convert a DataFrame of lines of text to a DataFrame of paragraphs.
    Paragraphs are defined as groups of lines that are separated by at least 1.5 times the average line height.
    """
    paragraphs = lines.groupby('paragraph_number').agg({'text': ' '.join})
    return paragraphs

In [271]:
lines = add_paragraph_number(lines)
paragraphs = lines_to_paragraphs(lines)

In [363]:
FIGURE_INDICATORS = ['figure', 'fig', 'fig.']
figure_line_numbers = lines.loc[
        lines['text'].str.lower().str.contains('|'.join(FIGURE_INDICATORS)), 'line_number']

In [367]:
# for each figure line number, get text from that line and the next 10 lines
figure_texts = []
for figure_line_number in figure_line_numbers:
    figure_line_set = lines.loc[
        (lines['line_number'] >= figure_line_number) &
        (lines['line_number'] < figure_line_number + 10)
    ]
    text = figure_line_set['text'].str.cat(sep=' ')
    figure_texts.append(text)

In [ ]:
figure_texts

In [ ]:
article.article_detail

In [389]:
jatsxml_string = requests.get(article.article_detail.jatsxml).text

In [390]:
from lxml import etree as ET

In [391]:
# Parse the XML string
root = ET.fromstring(jatsxml_string)

In [380]:
# Extract the body text
body_text = root.xpath("//body//p/text()")


In [392]:
from bs4 import BeautifulSoup

# Parse the XML string
soup = BeautifulSoup(jatsxml_string, 'lxml')

In [ ]:
soup.find_all('body')

In [397]:

# Extract the body element
body = soup.find('body')

# Extract all sec elements within the body element
sections = body.find_all('sec')

# Iterate through the sec elements and extract the title and p elements
for sec in sections:
    title = sec.find('title').text
    paragraphs = sec.find_all('p')
    body_text = [p.text for p in paragraphs]

    # Extract all fig elements within the body element
    figures = sec.find_all('fig')

    # Iterate through the fig elements and extract the caption element
    figure_captions = []
    for fig in figures:
        caption = fig.find('caption').text
        figure_captions.append(caption)

AttributeError: 'NoneType' object has no attribute 'text'

In [394]:
body_text = ''
figure_captions = []

for sec in soup.find_all('sec'):
    # Extract the title and body text of the section
    title = sec.title.text
    body_text += f'<h1>{title}</h1>'
    for p in sec.find_all('p'):
        body_text += p.text
    
    # Extract the captions of the figures in the section
    for fig in sec.find_all('fig'):
        caption = fig.caption.text
        figure_captions.append(caption)

print(body_text)
print(figure_captions)

AttributeError: 'NoneType' object has no attribute 'text'

In [384]:
body_text

''